In [1]:
import epics
from IPython.display import clear_output, update_display, Markdown, Image
import ipywidgets as widgets

from py4syn.epics.MotorClass import Motor


import pandas as pd
import numpy as np
import scipy as sp
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=True)

In [2]:
def display_callback(pvname=None, char_value='', **kw):
    update_display(pvname + " " + char_value, display_id=pvname)

In [3]:
def display_mark_callback(pvname=None, char_value='', **kw):
    update_display(Markdown("""|IOC:m1.DMOV|
            |------------|
            |"""+ char_value +"""|"""), display_id=pvname)

In [4]:
display("0", display_id="IOC:m1.RBV")
epics.camonitor("IOC:m1.RBV", callback=display_callback)

'0'

In [5]:
display("0", display_id="IOC:m1.DMOV")
epics.camonitor("IOC:m1.DMOV", callback=display_callback)


'0'

In [6]:
VAL_selection = widgets.BoundedFloatText(
    value=7.5,
    min=-200,
    max=200.0,
    step=0.1,
    description='VAL Target:',
    disabled=False
)

display(VAL_selection)

button = widgets.Button(description="Start Motor")
display(button)

def on_button_clicked(b):
    epics.caput("IOC:m1.VAL", VAL_selection.value)

button.on_click(on_button_clicked)



BoundedFloatText(value=7.5, description='VAL Target:', max=200.0, min=-200.0, step=0.1)

Button(description='Start Motor', style=ButtonStyle())

In [7]:
m1 = Motor("IOC:m1", "m1")

In [8]:
m1.setValue(20)

In [9]:
corrente = epics.PV("LNLS:ANEL:corrente")

In [10]:
corrente.get()

244.512

In [11]:
corrente.timestamp

1554897109.258579

In [12]:
def plot_current(current_data, last_range):
    # make figure
    figure = {
        'data': [],
        'layout': {},
    }

    # fill in most of layout
    figure['layout']['xaxis'] = {'range': last_range, 'title': 'Time (s)'}
    figure['layout']['yaxis'] = {'title': 'Corrent value (Amperes)', 'type': 'linear'}
    figure['layout']['hovermode'] = 'closest'

    # make data
    data_dict = {
        'x': list(current_data["time"]),
        'y': list(current_data["value"]),
        'mode': 'lines+markers',
    }
    figure['data'].append(data_dict)

    return go.FigureWidget(figure['data'], figure['layout'])
    

In [13]:
a = {"value": [20, 30, 40, 34], "time": [1, 2, 3, 4]}
g = plot_current(a, [0, 10])
display(g)


FigureWidget({
    'data': [{'mode': 'lines+markers',
              'type': 'scatter',
              'uid': '1…

In [43]:
initial_time = corrente.timestamp

plot_limit_time_range = 100
slider_limit_subdivision = 2

corrente_data = {"value": [], "time": []}
time_slider = [0]

g = plot_current(corrente_data, [time_slider[-1] - 100, time_slider[-1]] if time_slider[-1] > 100 else [0, 100])

def plot_data_callback(pvname=None, value=0, timestamp=0, **kwargs):
    with g.batch_update():
        if len(corrente_data["value"]) != 0 and value == corrente_data["value"][-1]:
            return

        corrente_data["value"].append(value)
        corrente_data["time"].append(timestamp - initial_time)

        if timestamp - initial_time > time_slider[-1] + plot_limit_time_range / slider_limit_subdivision:
            time_slider.append(round(timestamp - initial_time))
    
corrente = epics.PV("LNLS:ANEL:corrente", auto_monitor=True)
corrente.add_callback(plot_data_callback)

1

In [50]:
g

FigureWidget({
    'data': [{'mode': 'lines+markers',
              'type': 'scatter',
              'uid': '0…

In [49]:
corrente_data
# time_slider

{'time': [0.0,
  9.99993085861206,
  19.999979972839355,
  30.00002098083496,
  39.99986696243286],
 'value': [240.622, 240.573, 240.508, 240.459, 240.41]}